# GSMCnecator_GeneEssentiality_TnSeqValidation

In [2]:
import pandas as pd
import cobra
#from cameo import pfba
import json
from itertools import chain
from optlang.symbolics import Zero
from cobra.flux_analysis.variability import flux_variability_analysis
import numpy as np
from cobra.sampling import sample
from cobra.flux_analysis import pfba
import matplotlib.pyplot as plt
from cameo import models
from memote.support.consistency import detect_energy_generating_cycles
from bioservices import kegg

In [3]:
### load the model
cobra.core.configuration.Configuration.solver = 'cplex'
# load the iCN1361
m = cobra.io.read_sbml_model('../Model/iCN1361.xml')
m2 = cobra.io.read_sbml_model('Data/Jahn2022_files/RehMBEL1391_sbml_L3V1.xml')

# get the genes in the model
genes_iCN1361 = []
for i in m.genes:
    genes_iCN1361.append(i.id)
    
# get the genes in the model
genes_reh = []
for i in m2.genes:
    genes_reh.append(i.id)

### Gene essentiality in GSM function

In [71]:
def gene_essentiality(m, biomass):
    essential = []
    non_essential = []
    sol_wt = pfba(m)

    for ind, i in enumerate(m.genes):
        #print(ind, i.id)
        if i != 'Spontaneous':
            with m:
                m.genes.get_by_id(i.id).knock_out()
                try:
                    sol_ko = m.optimize()
                    if sol_ko.status != 'infeasible':
                        if sol_ko[biomass] > 0.05:
                            #print(sol_ko['R_Biomass'])
                            non_essential.append(i.id)
                        else:
                            essential.append(i.id)

                    else:
                        essential.append(i.id)

                except:
                    essential.append(i.id)
    return essential, non_essential
            

### load tn-seq data from Jahn 2021

In [4]:
tn_seq_lb = pd.read_excel('Data/Jahn2022_files/TnSeq_LBdata.xlsx')

In [5]:
# tn-seq data comparisons -  downloaded from Michael Jahn's ShinyLab
tn_seq = pd.read_csv('Data/Jahn2022_files/Cupriavidus_BarSeq_2021.csv')

### Fructose essentiality

In [8]:
fructose_continuous = tn_seq[tn_seq.condition == 'fructose - continuous']
fructose_continuous_T8 = fructose_continuous[fructose_continuous.timepoint == 8]

#### Calculate the median value of the replicates for each condition

In [61]:
fructose_scores_iCN1361 = {}
for i in genes_iCN1361:
    score = np.median(fructose_continuous_T8[fructose_continuous_T8.locus_tag == i].fitness_score)
    fructose_scores_iCN1361[i] = score
    
    
fructose_scores_reh = {}
for i in genes_reh:
    score = np.median(fructose_continuous_T8[fructose_continuous_T8.locus_tag == i].fitness_score)
    fructose_scores_reh[i] = score

/opt/anaconda2/envs/pyTFA_2/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/opt/anaconda2/envs/pyTFA_2/lib/python3.7/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [ ]:
tnseq_phenotypes_iCN1361 = {}
for ind, i in enumerate(tn_seq_lb['locus_tag']):
    
    # assign the essential genes
    if tn_seq_lb['essentiality'][ind] == 2:
        tnseq_phenotypes_iCN1361[i] = 2
    else:
        if i in fructose_scores_iCN1361.keys():
            if fructose_scores_iCN1361[i] < -3:
                pheno1 = 2
            else:
                pheno1 = 0
        else:
            pheno1 = 1
                       
        if pheno1 == 2:       
            tnseq_phenotypes_iCN1361[i] = 2
        elif tn_seq_lb['essentiality'][ind] == 0 or pheno1 == 0:
            tnseq_phenotypes_iCN1361[i] = 0
        else:
            tnseq_phenotypes_iCN1361[i] = 1
            
            
            
tnseq_phenotypes_reh = {}
for ind, i in enumerate(tn_seq_lb['locus_tag']):
    
    # assign the essential genes
    if tn_seq_lb['essentiality'][ind] == 2:
        tnseq_phenotypes_reh[i] = 2
    else:
        if i in fructose_scores_reh.keys():
            if fructose_scores_reh[i] < -3:
                pheno1 = 2
            else:
                pheno1 = 0
        else:
            pheno1 = 1
                       
        if pheno1 == 2:       
            tnseq_phenotypes_reh[i] = 2
        elif tn_seq_lb['essentiality'][ind] == 0 or pheno1 == 0:
            tnseq_phenotypes_reh[i] = 0
        else:
            tnseq_phenotypes_reh[i] = 1
        


        



#### Compare to the GSM

In [72]:
m.reactions.EX_BETA_D_FRUCTOSE_e.bounds = (-2.6, -2.6)
m.reactions.R_FRUPTS.bounds = (0.0, 0.0) # previous study found the ABC is the active reaction [ref]
m.objective = 'R_Biomass'

essential_fru, non_essential_fru = gene_essentiality(m, 'R_Biomass')
TP_fru, FP_fru, TN_fru, FN_fru, undefined = gene_essentiality_statistics(essential_fru, non_essential_fru, tnseq_phenotypes_iCN1361)


m2.reactions.EX_fru_e.bounds = (-2.6, -2.6)
m2.reactions.FRUpts.bounds = (0.0, 0.0) # previous study found the ABC is the active reaction [ref]
m2.reactions.FRUpts2.bounds = (0.0, 0.0)
m2.objective = 'Biomass'

essential_fru_reh, non_essential_fru_reh = gene_essentiality(m2, 'Biomass')
TP_fru2, FP_fru2, TN_fru2, FN_fru2, undefined2 = gene_essentiality_statistics(essential_fru_reh, non_essential_fru_reh, tnseq_phenotypes_reh)




/opt/anaconda2/envs/pyTFA_2/lib/python3.7/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


In [93]:
print('True positives: ', len(TP_fru))
print('False positives: ', len(FP_fru))
print('True negatives: ', len(TN_fru))
print('False negatives: ', len(FN_fru))
print('Undefined genes: ', len(undefined))
print('Overall accuracy iC1361: ', (len(TP_fru) + len(TN_fru))/(len(TP_fru) + len(TN_fru) + len(FN_fru) + len(FP_fru)))
print('Precision iCN1361: ', len(TP_fru)/(len(TP_fru)+ len(FP_fru)))
print('Recall iCN136: ', len(TP_fru)/(len(TP_fru) + len(FN_fru)))
print('')

print('True positives: ', len(TP_fru2))
print('False positives: ', len(FP_fru2))
print('True negatives: ', len(TN_fru2))
print('False negatives: ', len(FN_fru2))
print('Undefined genes: ', len(undefined2))
print('Overall accuracy Reh: ', (len(TP_fru2) + len(TN_fru2))/(len(TP_fru2) + len(TN_fru2) + len(FN_fru2) + len(FP_fru2)))
print('Precision Reh: ', len(TP_fru2)/(len(TP_fru2)+ len(FP_fru2)))
print('Recall Reh: ', len(TP_fru2)/(len(TP_fru2) + len(FN_fru2)))

True positives:  119
False positives:  41
True negatives:  1085
False negatives:  113
Undefined genes:  27
Overall accuracy iC1361:  0.8865979381443299
Precision iCN1361:  0.74375
Recall iCN136:  0.5129310344827587

True positives:  112
False positives:  47
True negatives:  1043
False negatives:  146
Undefined genes:  2
Overall accuracy Reh:  0.8568249258160238
Precision Reh:  0.7044025157232704
Recall Reh:  0.43410852713178294


### Succinate

In [74]:
succinate_continuous = tn_seq[tn_seq.condition == 'succinate - continuous']
succinate_continuous_T8 = succinate_continuous[succinate_continuous.timepoint == 8]

In [75]:
succinate_scores_iCN1361 = {}
for i in genes_iCN1361:
    score = np.median(succinate_continuous_T8[succinate_continuous_T8.locus_tag == i].fitness_score)
    succinate_scores_iCN1361[i] = score
    
    
succinate_scores_reh = {}
for i in genes_reh:
    score = np.median(succinate_continuous_T8[succinate_continuous_T8.locus_tag == i].fitness_score)
    succinate_scores_reh[i] = score

/opt/anaconda2/envs/pyTFA_2/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/opt/anaconda2/envs/pyTFA_2/lib/python3.7/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [88]:
tnseq_phenotypes_iCN1361_suc = {}
for ind, i in enumerate(tn_seq_lb['locus_tag']):
    
    # assign the essential genes
    if tn_seq_lb['essentiality'][ind] == 2:
        tnseq_phenotypes_iCN1361_suc[i] = 2
    else:
        if i in succinate_scores_iCN1361.keys():
            if succinate_scores_iCN1361[i] < -3:
                pheno1 = 2
            else:
                pheno1 = 0
        else:
            pheno1 = 1
                       
        if pheno1 == 2:       
            tnseq_phenotypes_iCN1361[i] = 2
        elif tn_seq_lb['essentiality'][ind] == 0 or pheno1 == 0:
            tnseq_phenotypes_iCN1361_suc[i] = 0
        else:
            tnseq_phenotypes_iCN1361_suc[i] = 1
            
            
            
tnseq_phenotypes_reh_suc = {}
for ind, i in enumerate(tn_seq_lb['locus_tag']):
    
    # assign the essential genes
    if tn_seq_lb['essentiality'][ind] == 2:
        tnseq_phenotypes_reh_suc[i] = 2
    else:
        if i in succinate_scores_reh.keys():
            if succinate_scores_reh[i] < -3:
                pheno1 = 2
            else:
                pheno1 = 0
        else:
            pheno1 = 1
                       
        if pheno1 == 2:       
            tnseq_phenotypes_reh_suc[i] = 2
        elif tn_seq_lb['essentiality'][ind] == 0 or pheno1 == 0:
            tnseq_phenotypes_reh_suc[i] = 0
        else:
            tnseq_phenotypes_reh_suc[i] = 1
        


        




In [89]:
m.reactions.EX_BETA_D_FRUCTOSE_e.bounds = (0.0, 0.0)
m.reactions.EX_SUC_e.bounds = (-10.0, 0.0)
m.objective = 'R_Biomass'

essential_suc, non_essential_suc = gene_essentiality(m, 'R_Biomass')
TP_suc, FP_suc, TN_suc, FN_suc, undefined = gene_essentiality_statistics(essential_suc, non_essential_suc, tnseq_phenotypes_iCN1361_suc)


m2.reactions.EX_fru_e.bounds = (0.0, 0.0)
m2.reactions.EX_succ_e.bounds = (-10.0, 0.0)
m2.objective = 'Biomass'

essential_suc_reh, non_essential_suc_reh = gene_essentiality(m2, 'Biomass')
TP_suc2, FP_suc2, TN_suc2, FN_suc2, undefined2 = gene_essentiality_statistics(essential_suc_reh, non_essential_suc_reh, tnseq_phenotypes_reh_suc)





In [91]:
print('True positives: ', len(TP_suc))
print('False positives: ', len(FP_fru))
print('True negatives: ', len(TN_suc))
print('False negatives: ', len(FN_suc))
print('Undefined genes: ', len(undefined))

print('Overall accuracy iC1361: ', (len(TP_suc) + len(TN_suc))/(len(TP_suc) + len(TN_suc) + len(FN_suc) + len(FP_suc)))
print('Precision iCN1361: ', len(TP_suc)/(len(TP_suc)+ len(FP_suc)))
print('Recall iCN136: ', len(TP_suc)/(len(TP_suc) + len(FN_suc)))
print('')

print('True positives: ', len(TP_suc2))
print('False positives: ', len(FP_suc2))
print('True negatives: ', len(TN_suc2))
print('False negatives: ', len(FN_suc2))
print('Undefined genes: ', len(undefined2))

print('Overall accuracy RehMBEL: ', (len(TP_suc2) + len(TN_suc2))/(len(TP_suc2) + len(TN_suc2) + len(FN_suc2) + len(FP_suc2)))
print('Precision RehMBEL: ', len(TP_suc2)/(len(TP_suc2)+ len(FP_suc2)))
print('Recall RehMBEL: ', len(TP_suc2)/(len(TP_suc2) + len(FN_suc2)))

True positives:  101
False positives:  41
True negatives:  1093
False negatives:  100
Undefined genes:  27
Overall accuracy iC1361:  0.8943820224719101
Precision iCN1361:  0.7112676056338029
Recall iCN136:  0.5024875621890548

True positives:  109
False positives:  51
True negatives:  1050
False negatives:  138
Undefined genes:  2
Overall accuracy RehMBEL:  0.8597922848664689
Precision RehMBEL:  0.68125
Recall RehMBEL:  0.44129554655870445


### Formate

In [94]:
formate_continuous = tn_seq[tn_seq.condition == 'formate - continuous']
formate_continuous_T8 = formate_continuous[formate_continuous.timepoint == 8]

In [95]:
formate_scores_iCN1361 = {}
for i in genes_iCN1361:
    score = np.median(formate_continuous_T8[formate_continuous_T8.locus_tag == i].fitness_score)
    formate_scores_iCN1361[i] = score
    
    
formate_scores_reh = {}
for i in genes_reh:
    score = np.median(formate_continuous_T8[formate_continuous_T8.locus_tag == i].fitness_score)
    formate_scores_reh[i] = score

/opt/anaconda2/envs/pyTFA_2/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/opt/anaconda2/envs/pyTFA_2/lib/python3.7/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [96]:
tnseq_phenotypes_iCN1361_for = {}
for ind, i in enumerate(tn_seq_lb['locus_tag']):
    
    # assign the essential genes
    if tn_seq_lb['essentiality'][ind] == 2:
        tnseq_phenotypes_iCN1361_for[i] = 2
    else:
        if i in formate_scores_iCN1361.keys():
            if formate_scores_iCN1361[i] < -3:
                pheno1 = 2
            else:
                pheno1 = 0
        else:
            pheno1 = 1
                       
        if pheno1 == 2:       
            tnseq_phenotypes_iCN1361[i] = 2
        elif tn_seq_lb['essentiality'][ind] == 0 or pheno1 == 0:
            tnseq_phenotypes_iCN1361_for[i] = 0
        else:
            tnseq_phenotypes_iCN1361_for[i] = 1
            
            
            
tnseq_phenotypes_reh_for = {}
for ind, i in enumerate(tn_seq_lb['locus_tag']):
    
    # assign the essential genes
    if tn_seq_lb['essentiality'][ind] == 2:
        tnseq_phenotypes_reh_for[i] = 2
    else:
        if i in formate_scores_reh.keys():
            if formate_scores_reh[i] < -3:
                pheno1 = 2
            else:
                pheno1 = 0
        else:
            pheno1 = 1
                       
        if pheno1 == 2:       
            tnseq_phenotypes_reh_for[i] = 2
        elif tn_seq_lb['essentiality'][ind] == 0 or pheno1 == 0:
            tnseq_phenotypes_reh_for[i] = 0
        else:
            tnseq_phenotypes_reh_for[i] = 1
        


        





In [101]:
m.reactions.EX_BETA_D_FRUCTOSE_e.bounds = (0.0, 0.0)
m.reactions.EX_SUC_e.bounds = (0.0, 0.0)
m.reactions.EX_FORMATE_e.bounds = (-20.0, 0.0)
m.reactions.EX_PROTON_e.bounds = (-1000.0, 1000.0)
m.objective = 'R_Biomass'

essential_for, non_essential_for = gene_essentiality(m, 'R_Biomass')
TP_for, FP_for, TN_for, FN_for, undefined = gene_essentiality_statistics(essential_for, non_essential_for, tnseq_phenotypes_iCN1361_for)


m2.reactions.EX_fru_e.bounds = (0.0, 0.0)
m2.reactions.EX_succ_e.bounds = (0.0, 0.0)
m2.reactions.EX_formate_e.bounds = (-20.0, 0.0)
m2.objective = 'Biomass'

essential_for_reh, non_essential_for_reh = gene_essentiality(m2, 'Biomass')
TP_for2, FP_for2, TN_for2, FN_for2, undefined2 = gene_essentiality_statistics(essential_for_reh, non_essential_for_reh, tnseq_phenotypes_reh_for)






In [102]:
print('True positives: ', len(TP_for))
print('False positives: ', len(FP_for))
print('True negatives: ', len(TN_for))
print('False negatives: ', len(FN_for))
print('Undefined genes: ', len(undefined))

print('Overall accuracy iC1361: ', (len(TP_for) + len(TN_for))/(len(TP_for) + len(TN_for) + len(FN_for) + len(FP_for)))
print('Precision iCN1361: ', len(TP_for)/(len(TP_for)+ len(FP_for)))
print('Recall iCN136: ', len(TP_for)/(len(TP_for) + len(FN_for)))
print('')

print('True positives: ', len(TP_for2))
print('False positives: ', len(FP_for2))
print('True negatives: ', len(TN_for2))
print('False negatives: ', len(FN_for2))
print('Undefined genes: ', len(undefined2))

print('Overall accuracy RehMBEL: ', (len(TP_for2) + len(TN_for2))/(len(TP_for2) + len(TN_for2) + len(FN_for2) + len(FP_for2)))
print('Precision RehMBEL: ', len(TP_for2)/(len(TP_for2)+ len(FP_for2)))
print('Recall RehMBEL: ', len(TP_for2)/(len(TP_for2) + len(FN_for2)))

True positives:  105
False positives:  43
True negatives:  1078
False negatives:  96
Undefined genes:  40
Overall accuracy iC1361:  0.8948562783661119
Precision iCN1361:  0.7094594594594594
Recall iCN136:  0.5223880597014925

True positives:  113
False positives:  52
True negatives:  1036
False negatives:  147
Undefined genes:  2
Overall accuracy RehMBEL:  0.8523738872403561
Precision RehMBEL:  0.6848484848484848
Recall RehMBEL:  0.4346153846153846


### Save phenotypes as csv file for MEMOTE

In [216]:
# save to csv for MEMOTE 
df_ess_tnseq_fru = pd.DataFrame()

gene = []
essential = []
comment = []
for i, j in tnseq_phenotypes_iCN1361.items():
    if str(i) != 'nan':
        if i in m.genes:
    
            if j == 2:
                gene.append(i)
                essential.append('yes')
                comment.append('')
            elif j == 0:
                gene.append(i)
                essential.append('no')
                comment.append('')
                
df_ess_tnseq_fru.insert(0, column = 'gene', value = gene)
df_ess_tnseq_fru.insert(1, column = 'essential', value = essential)
df_ess_tnseq_fru.insert(2, column = 'comment', value = comment)

df_ess_tnseq_fru.to_csv('../MEMOTE_repo/sbrc_cnecator_gsm/data/essentiality/knockouts_tnseq_fru.csv')


In [217]:
# save to csv for MEMOTE 
df_ess_tnseq_suc = pd.DataFrame()

gene = []
essential = []
comment = []
for i, j in tnseq_phenotypes_iCN1361_suc.items():
    if str(i) != 'nan':
        if i in m.genes:
    
            if j == 2:
                gene.append(i)
                essential.append('yes')
                comment.append('')
            elif j == 0:
                gene.append(i)
                essential.append('no')
                comment.append('')
                
df_ess_tnseq_suc.insert(0, column = 'gene', value = gene)
df_ess_tnseq_suc.insert(1, column = 'essential', value = essential)
df_ess_tnseq_suc.insert(2, column = 'comment', value = comment)

df_ess_tnseq_suc.to_csv('../MEMOTE_repo/sbrc_cnecator_gsm/data/essentiality/knockouts_tnseq_suc.csv')


In [220]:
# save to csv for MEMOTE 
df_ess_tnseq_for = pd.DataFrame()

gene = []
essential = []
comment = []
for i, j in tnseq_phenotypes_iCN1361_for.items():
    if str(i) != 'nan':
        if i in m.genes:
    
            if j == 2:
                gene.append(i)
                essential.append('yes')
                comment.append('')
            elif j == 0:
                gene.append(i)
                essential.append('no')
                comment.append('')
                
df_ess_tnseq_for.insert(0, column = 'gene', value = gene)
df_ess_tnseq_for.insert(1, column = 'essential', value = essential)
df_ess_tnseq_for.insert(2, column = 'comment', value = comment)

df_ess_tnseq_for.to_csv('../MEMOTE_repo/sbrc_cnecator_gsm/data/essentiality/knockouts_tnseq_for.csv')
